In [6]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import cross_val_score
from sklearn.metrics import make_scorer
from sklearn.metrics import roc_auc_score
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import random

Import the data

In [2]:
data = pd.read_csv('training_v2.csv')
X = data.drop('hospital_death', axis=1)
y = data.hospital_death

Create the model we will use

In [3]:
from xgboost import XGBClassifier
xboost_c = XGBClassifier(colsample_bytree=0.3, eta=0.02, gamma = 0.1, max_depth = 6, min_child_weight = 6)
xboost = XGBClassifier()

tester()
- This function will do all the preprocessing and model evaluation

In [4]:
# write testing function
def tester(X, y, model):
    X_fill = X.fillna(-1)
    cols = X.columns
    num_cols = X._get_numeric_data().columns
    cat_cols = list(set(cols) - set(num_cols))
    X_fill[cat_cols] = X_fill[cat_cols].astype('str')
    # label encoding for categorical features
    for col in cat_cols:
        le = LabelEncoder().fit(np.unique(X_fill[col].unique().tolist()))
        X_fill[col] = le.transform(X_fill[col])+1
      #split and train
    X_train, X_test, y_train, y_test = train_test_split(X_fill, y, stratify = y, 
                                            test_size=0.2, random_state=42)
    model.fit(X_train, y_train)
    preds = model.predict_proba(X_test)
      #score
    score = roc_auc_score(y_test, preds[:,1:])
    return score

In [5]:
tester(X, y, xboost_c)

0.9089127543997455

FindImportantFeats()
- Quickly samples k number of features (sample size) and checks auc score with only these features. You can specify the number of iterations and the sample size of your random features
- Supposedly, with enough iterations you'll find the optimal set of features (for the given feature sample size)

1. Working on a master_list to return the features that give auc above a certain threshold. I suspect that features that frequently score high are most useful.


In [7]:
def FindImportantFeats(X, y, model, sample_size = 40, iters=10):
    best_feats = []
    master_list = pd.Series()
    best_score = 0.0
    for i in range(iters):
        random_features = random.sample(list(X.columns), sample_size)
        auc_score = float(tester(X[random_features], y, model))
        print('Iteration:', i+1, '---', auc_score)
        if auc_score > 0.8:
            rf = pd.Series(random_features)
            master_list = master_list.append(rf, ignore_index=True)
        if auc_score > best_score:
            print('found a good score')
            best_score = auc_score
            best_feats = random_features
    print(best_score)
    return best_feats

In [ ]:
bestfeats = FindImportantFeats(X,y,xboost_c, sample_size=150, iters=100)
bestfeats